In [1]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import load
from sklearn.metrics import classification_report

<Figure size 1000x600 with 0 Axes>

In [2]:
wesad = load.WESAD()
df = wesad.separate_and_feature_extract(sample_n=420000) #700 * 60 * 10 (10mins)

Using ready-made DataFrame


In [3]:
df.shape

(3540, 20)

In [4]:
y = df['label'].to_numpy()-1
X = df.drop(['label','subject'],axis=1)
X = X.to_numpy()

X = X.reshape(*X.shape, 1)  # 轉回 CNN 需要的格式

In [5]:
# 分割訓練測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# 建立 1D CNN 模型
def build_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(4, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(8, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [7]:
def build_cnn_lstm(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(4, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(8, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.LSTM(16, return_sequences=False))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [8]:
# 初始化模型
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = len(np.unique(y))
model = build_cnn(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 10
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# 評估模型
loss, acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc * 100:.2f}%')

Epoch 1/10


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5612 - loss: 1.3880 - val_accuracy: 0.6874 - val_loss: 0.6038
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7322 - loss: 0.5736 - val_accuracy: 0.7137 - val_loss: 0.5306
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7532 - loss: 0.5083 - val_accuracy: 0.8362 - val_loss: 0.4747
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8114 - loss: 0.4512 - val_accuracy: 0.8729 - val_loss: 0.4263
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8284 - loss: 0.4169 - val_accuracy: 0.7881 - val_loss: 0.3898
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8026 - loss: 0.3766 - val_accuracy: 0.7976 - val_loss: 0.3666
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8379 - loss: 0.3419 - val_accuracy: 0.8362 - val_loss: 0.3453
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8361 - loss: 0.3266 - val_accuracy: 0.8606 - val_loss: 0.3241
Epo

In [9]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           1       0.83      0.92      0.88       555
           2       0.90      0.80      0.85       507

    accuracy                           0.86      1062
   macro avg       0.87      0.86      0.86      1062
weighted avg       0.87      0.86      0.86      1062



In [10]:
# 初始化模型
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = len(np.unique(y))
model = build_cnn_lstm(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 10
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# 評估模型
loss, acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc * 100:.2f}%')

Epoch 1/10


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5052 - loss: 0.6865 - val_accuracy: 0.5819 - val_loss: 0.6677
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6243 - loss: 0.6519 - val_accuracy: 0.6864 - val_loss: 0.5936
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7566 - loss: 0.5618 - val_accuracy: 0.7241 - val_loss: 0.5177
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8202 - loss: 0.4657 - val_accuracy: 0.8672 - val_loss: 0.3814
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8668 - loss: 0.3726 - val_accuracy: 0.9021 - val_loss: 0.3268
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8886 - loss: 0.3263 - val_accuracy: 0.8710 - val_loss: 0.3027
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8874 - loss: 0.2915 - val_accuracy: 0.9040 - val_loss: 0.2903
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9085 - loss: 0.2701 - val_accuracy: 0.9209 - val_loss: 0.2548
Epo

In [11]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
              precision    recall  f1-score   support

           1       0.89      0.98      0.93       555
           2       0.97      0.87      0.92       507

    accuracy                           0.93      1062
   macro avg       0.93      0.93      0.93      1062
weighted avg       0.93      0.93      0.93      1062

